In [1]:
#pip install msedge-selenium-tools selenium==3.141
#pip install selenium

from time import sleep
import csv
import os
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
from dotenv import load_dotenv
from datetime import date


#function to collect the data per tweet
def get_tweet_data(card):
    #EXTRACT DATA FROM TWEET
    twitter_username = card.find_element_by_xpath('.//span').text
    twitter_account = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    #Some 'cards' are ads and dont have a time
    try:
        tweet_date = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NuSuchElementException:
        return
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    reply = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    tweet_content = comment + reply
    tweet_reply_count = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    tweet_retweet_count = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    tweet_like_count = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (twitter_username, twitter_account, tweet_date, tweet_content, tweet_reply_count, tweet_retweet_count, tweet_like_count)
    return tweet


#loading login credentials from .env
#open Chrome and navigate to Twitter Login screen
load_dotenv()

twitter_username = os.getenv("TWITTER_USERNAME")
twitter_password = os.getenv("TWITTER_PASSWORD")

driver = Chrome(executable_path='/Users/christiankunz/Downloads/chromedriver')

driver.get('https://twitter.com/login')
sleep(5)


#setup Search criteria
twitter_from = 'elonmusk' #twitter account
twitter_since = '2015-01-01' #start date of tweets
twitter_until = str(date.today()) #gets current date

search = '(from:'+twitter_from+') until:'+twitter_until+' since:'+ twitter_since


#login into Twitter
username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys(twitter_username)

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(twitter_password)

password.send_keys(Keys.RETURN)
sleep(5)


#search within Twitter for our 'tag'
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys(search)
search_input.send_keys(Keys.RETURN)
sleep(10)


#navigate to historical tab ('Latest')
driver.find_element_by_link_text('Latest').click()
sleep(5)


#Twitters website is dynamic and Tweets appear after continoiusly scrolling
#need to scrap tweets, scroll further, scrap more tweets... Need to ensure we don't copy same tweets multiple times
tweet_data = []
tweet_ids = set()
last_position = driver.execute_script('return window.pageYOffset;')
scrolling = True

while scrolling:
    cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')

    for card in cards:
        data = get_tweet_data(card)
        if data:
            tweet_id = ''.join(data)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                tweet_data.append(data)
                
    scroll_attempt = 0
    while True:
        #check scroll position and scroll
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(5)
        current_position = driver.execute_script('return window.pageYOffset;')
        if last_position == current_position:
            scroll_attempt += 1
            
            #reached end of scroll
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(5)
        else:
            last_position = current_position
            break


In [2]:
#save tweet list into DataFrame

import csv
with open("data/tweets_elonmusk_raw_FINAL.csv", "w") as new_file:
    header = [
        'twitter_username',
        'twitter_account',
        'tweet_date',
        'tweet_content',
        'tweet_reply_count',
        'tweet_retweet_count',
        'tweet_like_count'     
    ]
    writer = csv.writer(new_file)
    writer.writerow(header)
    writer.writerows(tweet_data)